In [ ]:
#Copyright 2024 Vraj Shah, Thomas Parashos, and Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import os
# os.system('pip install keras')
import sys, imp, math, random, time
import numpy as np
import pandas as pd
import csv
import random, copy
from pathlib import Path
from IPython.display import display

from io import StringIO
from sklearn import tree
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# from sklearn.ensemble import RandomForestClassifier
from downstream_model_CorrectEncoding_1split import *
from Featurize_CorrectEncoding import *

from matplotlib import pyplot as plt
from random import choices

import warnings
warnings.filterwarnings('ignore')

import ast
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

plt.rcParams.update({'font.size': 26})
plt.rcParams.update({'figure.autolayout': True})

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

np.random.seed(100)

In [ ]:
def getFileInfoMLP(file_lst):
    totnos = 1
    index = 0
#     a.append(2)
#     a.append(5)    
    for file in file_lst:
#         print('==============' + str(index) + ':' + str(nrset[index]) + '==============')
        LRtrain_lst, LRval_lst, LRtest_lst = [], [], []
        RFtrain_lst, RFval_lst, RFtest_lst = [], [], []        
        gen_original_LR,gen_original_RF,gen_duplicate_LR,gen_duplicate_RF = [],[],[],[]
        inc_var_LR,inc_var_RF = [],[]        
        
        data = file.read()

        x = data.split('\n')

        curindex = 4
        while curindex < 39*NUMRUNS:
#             print(curindex)
            try:
                scrtr_LR, scrval_LR, scrte_LR = float(x[curindex-2]), float(x[curindex-1]), float(x[curindex])
#                 scrtr_RF, scrval_RF, scrte_RF = float(x[curindex+2]),float(x[curindex+3]),float(x[curindex+4])

#                 curindex = curindex+1
                LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+15]),ast.literal_eval(x[curindex+16]),ast.literal_eval(x[curindex+17])
#                 RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+19]),ast.literal_eval(x[curindex+20]),ast.literal_eval(x[curindex+21])

                gen_error_LR = (scrtr_LR-scrval_LR)
                gen_original_LR.extend([gen_error_LR])
#                 gen_original_RF.extend([gen_error_RF])                
        
#                 print('***' + str(len(RFtest)))
#                 print(RFtest)
    #             LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+25]),ast.literal_eval(x[curindex+26]),ast.literal_eval(x[curindex+27])
    #             print(ast.literal_eval(x[curindex+28]))
    #             RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+29]),ast.literal_eval(x[curindex+30]),ast.literal_eval(x[curindex+31])

                diff_train_lst_LR_full = [round(scrtr_LR - number,3) for number in LRtrain]
                diff_lst_LR_full = [round(scrval_LR - number,3) for number in LRval] 
                diff_hld_lst_LR_full = [round(scrte_LR - number,3) for number in LRtest]

#                 diff_train_lst_RF_full = [round(scrtr_RF - number,3) for number in RFtrain]
#                 diff_lst_RF_full = [round(scrval_RF - number,3) for number in RFval]
#                 diff_hld_lst_RF_full = [round(scrte_RF - number,3) for number in RFtest]


                LRtrain_lst.extend(diff_train_lst_LR_full)
                LRval_lst.extend(diff_lst_LR_full)
                LRtest_lst.extend(diff_hld_lst_LR_full)

#                 RFtrain_lst.extend(diff_train_lst_RF_full)
#                 RFval_lst.extend(diff_lst_RF_full)
#                 RFtest_lst.extend(diff_hld_lst_RF_full)
                
                gen_duplicate_LR.extend([round(LRtrain[pq] - LRval[pq],3) for pq in range(len(LRtrain))])
#                 gen_duplicate_RF.extend([round(RFtrain[pq] - RFval[pq],3) for pq in range(len(RFtrain))])                

                inc_var_LR.extend([round(LRtrain[pq] - LRval[pq]-gen_error_LR,3) for pq in range(len(LRtrain))])
#                 inc_var_RF.extend([round(RFtrain[pq] - RFval[pq]-gen_error_RF,3) for pq in range(len(RFtrain))])
                                
            except: flag = 1
            curindex = curindex + 39                
            
        LRtest_lst_final.append(LRtest_lst)
#         RFtest_lst_final.append(RFtest_lst)

        gen_original_LR_final.append(gen_original_LR)
#         gen_original_RF_final.append(gen_original_RF)
        gen_duplicate_LR_final.append(gen_duplicate_LR)
#         gen_duplicate_RF_final.append(gen_duplicate_RF)        
        inc_var_LR_final.append(inc_var_LR)
        inc_var_RF_final.append(inc_var_LR)
        
#         print(len(LRtest_lst_final))
#         print(totnos)
#         print(len(RFtest_lst_final[index]))
        print(len(LRtest_lst_final[index]))
        totnos += 1
        
        LR_min_lst.append(np.min(LRtest_lst_final[index]))
        LR_perc25_lst.append(np.quantile(LRtest_lst_final[index], .25))
        LR_median_lst.append(np.median(LRtest_lst_final[index]))
        LR_perc75_lst.append(np.quantile(LRtest_lst_final[index], .75))
        LR_max_lst.append(np.max(LRtest_lst_final[index]))

        inc_RF_min_lst.append(np.min(inc_var_RF_final[index]))
        inc_RF_perc25_lst.append(np.quantile(inc_var_RF_final[index], .25))
        inc_RF_median_lst.append(np.median(inc_var_RF_final[index]))
        inc_RF_perc75_lst.append(np.quantile(inc_var_RF_final[index], .75))
        inc_RF_max_lst.append(np.max(inc_var_RF_final[index]))           
        
        index = index +1
     
####################################################################################################################################    

def getFileInfo(file_lst):
    totnos = 1
    index = 0
#     a.append(2)
#     a.append(5)    
    for file in file_lst:
#         print('==============' + str(index) + ':' + str(nrset[index]) + '==============')
        LRtrain_lst, LRval_lst, LRtest_lst = [], [], []
        RFtrain_lst, RFval_lst, RFtest_lst = [], [], []        
        gen_original_LR,gen_original_RF,gen_duplicate_LR,gen_duplicate_RF = [],[],[],[]
        inc_var_LR,inc_var_RF = [],[]        
        
        data = file.read()

        x = data.split('\n')

        curindex = 4
        while curindex < 39*NUMRUNS:
#             print(curindex)
            try:
                scrtr_LR, scrval_LR, scrte_LR = float(x[curindex-2]), float(x[curindex-1]), float(x[curindex])
                scrtr_RF, scrval_RF, scrte_RF = float(x[curindex+2]),float(x[curindex+3]),float(x[curindex+4])

#                 curindex = curindex+1
                LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+15]),ast.literal_eval(x[curindex+16]),ast.literal_eval(x[curindex+17])
                RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+19]),ast.literal_eval(x[curindex+20]),ast.literal_eval(x[curindex+21])

                gen_error_LR, gen_error_RF = (scrtr_LR-scrval_LR),(scrtr_RF-scrval_RF)
                gen_original_LR.extend([gen_error_LR])
                gen_original_RF.extend([gen_error_RF])                
        
#                 print('***' + str(len(RFtest)))
#                 print(RFtest)
    #             LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+25]),ast.literal_eval(x[curindex+26]),ast.literal_eval(x[curindex+27])
    #             print(ast.literal_eval(x[curindex+28]))
    #             RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+29]),ast.literal_eval(x[curindex+30]),ast.literal_eval(x[curindex+31])

                diff_train_lst_LR_full = [round(scrtr_LR - number,3) for number in LRtrain]
                diff_lst_LR_full = [round(scrval_LR - number,3) for number in LRval] 
                diff_hld_lst_LR_full = [round(scrte_LR - number,3) for number in LRtest]

                diff_train_lst_RF_full = [round(scrtr_RF - number,3) for number in RFtrain]
                diff_lst_RF_full = [round(scrval_RF - number,3) for number in RFval]
                diff_hld_lst_RF_full = [round(scrte_RF - number,3) for number in RFtest]


                LRtrain_lst.extend(diff_train_lst_LR_full)
                LRval_lst.extend(diff_lst_LR_full)
                LRtest_lst.extend(diff_hld_lst_LR_full)

                RFtrain_lst.extend(diff_train_lst_RF_full)
                RFval_lst.extend(diff_lst_RF_full)
                RFtest_lst.extend(diff_hld_lst_RF_full)
                
                gen_duplicate_LR.extend([round(LRtrain[pq] - LRval[pq],3) for pq in range(len(LRtrain))])
                gen_duplicate_RF.extend([round(RFtrain[pq] - RFval[pq],3) for pq in range(len(RFtrain))])                

                inc_var_LR.extend([round(LRtrain[pq] - LRval[pq]-gen_error_LR,3) for pq in range(len(LRtrain))])
                inc_var_RF.extend([round(RFtrain[pq] - RFval[pq]-gen_error_RF,3) for pq in range(len(RFtrain))])
                                
            except: flag = 1
            curindex = curindex + 39                
            
        LRtest_lst_final.append(LRtest_lst)
        RFtest_lst_final.append(RFtest_lst)

        gen_original_LR_final.append(gen_original_LR)
        gen_original_RF_final.append(gen_original_RF)
        gen_duplicate_LR_final.append(gen_duplicate_LR)
        gen_duplicate_RF_final.append(gen_duplicate_RF)        
        inc_var_LR_final.append(inc_var_LR)
        inc_var_RF_final.append(inc_var_RF)        
        
#         print(len(LRtest_lst_final))
#         print(totnos)
        print(len(RFtest_lst_final[index]))
        totnos += 1
        
        RF_min_lst.append(np.min(RFtest_lst_final[index]))
        RF_perc25_lst.append(np.quantile(RFtest_lst_final[index], .25))
        RF_median_lst.append(np.median(RFtest_lst_final[index]))
        RF_perc75_lst.append(np.quantile(RFtest_lst_final[index], .75))
        RF_max_lst.append(np.max(RFtest_lst_final[index]))
        
        
        inc_RF_min_lst.append(np.min(inc_var_RF_final[index]))
        inc_RF_perc25_lst.append(np.quantile(inc_var_RF_final[index], .25))
        inc_RF_median_lst.append(np.median(inc_var_RF_final[index]))
        inc_RF_perc75_lst.append(np.quantile(inc_var_RF_final[index], .75))
        inc_RF_max_lst.append(np.max(inc_var_RF_final[index]))        
        
        index = index +1
     
#     return RFtest_lst_final
  
####################################################################################################################################

def getFileInfoH2o(file_lst):
    totnos = 1
    index = 0
    for file in file_lst:
#         print('==============' + str(index) + ':' + str(nrset[index]) + '==============')
        LRtrain_lst, LRval_lst, LRtest_lst = [], [], []
        RFtrain_lst, RFval_lst, RFtest_lst = [], [], []        
        gen_original_LR,gen_original_RF,gen_duplicate_LR,gen_duplicate_RF = [],[],[],[]
        inc_var_LR,inc_var_RF = [],[]        
        
        data = file.read()

        x = data.split('\n')

        curindex = 4
        while curindex < 27*NUMRUNS:
#             print(curindex)
            try:
                scrtr_RF, scrval_RF, scrte_RF = float(x[curindex-2]), float(x[curindex-1]), float(x[curindex])
#                 scrtr_RF, scrval_RF, scrte_RF = float(x[curindex+2]),float(x[curindex+3]),float(x[curindex+4])

#                 curindex = curindex+1
                RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+11]),ast.literal_eval(x[curindex+12]),ast.literal_eval(x[curindex+13])
#                 RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+19]),ast.literal_eval(x[curindex+20]),ast.literal_eval(x[curindex+21])

                gen_error_RF = (scrtr_RF-scrval_RF)
                gen_original_RF.extend([gen_error_RF])
            

                diff_train_lst_RF_full = [round(scrtr_RF - number,3) for number in RFtrain]
                diff_lst_RF_full = [round(scrval_RF - number,3) for number in RFval]
                diff_hld_lst_RF_full = [round(scrte_RF - number,3) for number in RFtest]


#                 LRtrain_lst.extend(diff_train_lst_LR_full)
#                 LRval_lst.extend(diff_lst_LR_full)
#                 LRtest_lst.extend(diff_hld_lst_LR_full)

                RFtrain_lst.extend(diff_train_lst_RF_full)
                RFval_lst.extend(diff_lst_RF_full)
                RFtest_lst.extend(diff_hld_lst_RF_full)
                
#                 gen_duplicate_LR.extend([round(LRtrain[pq] - LRval[pq],3) for pq in range(len(LRtrain))])
                gen_duplicate_RF.extend([round(RFtrain[pq] - RFval[pq],3) for pq in range(len(RFtrain))])                

#                 inc_var_LR.extend([round(LRtrain[pq] - LRval[pq]-gen_error_LR,3) for pq in range(len(LRtrain))])
                inc_var_RF.extend([round(RFtrain[pq] - RFval[pq]-gen_error_RF,3) for pq in range(len(RFtrain))])
                                
            except: flag = 1
            curindex = curindex + 27                
            
#         LRtest_lst_final.append(LRtest_lst)
        RFtest_lst_final.append(RFtest_lst)

#         gen_original_LR_final.append(gen_original_LR)
        gen_original_RF_final.append(gen_original_RF)
#         gen_duplicate_LR_final.append(gen_duplicate_LR)
        gen_duplicate_RF_final.append(gen_duplicate_RF)        
#         inc_var_LR_final.append(inc_var_LR)
        inc_var_RF_final.append(inc_var_RF)        
        
#         print(len(LRtest_lst_final))
#         print(totnos)
        print(len(RFtest_lst_final[index]))
        totnos += 1
        
        RF_min_lst.append(np.min(RFtest_lst_final[index]))
        RF_perc25_lst.append(np.quantile(RFtest_lst_final[index], .25))
        RF_median_lst.append(np.median(RFtest_lst_final[index]))
        RF_perc75_lst.append(np.quantile(RFtest_lst_final[index], .75))
        RF_max_lst.append(np.max(RFtest_lst_final[index]))
        
        index = index +1
     
#     return LRtest_lst_final
    

Choose the ML model and encoding method to analyze

[Option 1] High Bias Random Forest with One-hot encoding [Run below cell]

In [ ]:
ish2o = 0
ismlp = 0
curcol = 'duplicateColumn'
MODEL_TYPE = 'allxRF-BiasDT'

[Option 2] High Variance Random Forest with One-hot encoding

In [ ]:
ish2o = 0
ismlp = 0
curcol = 'duplicateColumn'
MODEL_TYPE = 'allxRF-Var'

[Option 3] High Bias H2o or Random Forest String encoding method

In [ ]:
### 
ish2o = 1
ismlp = 0
curcol = 'h2o-duplicateColumn'
MODEL_TYPE = 'allxH2o-Bias'

[Option 4] High Variance H2o or Random Forest String encoding method

In [2]:
ish2o = 1
ismlp = 0
curcol = 'h2o-duplicateColumn'
MODEL_TYPE = 'allxH2o-Var'

[Option 5] High Bias MLP with One-hot encoding method

In [ ]:
ish2o = 0
ismlp = 1
curcol = 'duplicateColumn'

MODEL_TYPE = 'allxMLP-Bias'

[Option 6] High Variance MLP with One-hot encoding method

In [ ]:
### High Variance MLP
ish2o = 0
ismlp = 1
curcol = 'duplicateColumn'
MODEL_TYPE = 'allxMLP-Var'

[Option 6] SVM with One-hot encoding method

In [ ]:
ismlp = 0
ish2o = 0
MODEL_TYPE = 'svm'
curcol = 'duplicateColumn'

[Option 7] XGBoost with One-hot encoding method

ismlp = 0
ish2o = 0
MODEL_TYPE = 'xgb'
curcol = 'duplicateColumn'

[Option 8] To observe the impact of noisy non-relevant column on ML, Run the below cell

In [ ]:
curcol = 'noiseColumn'

# Vary Train Examples

In [ ]:

LRtest_lst_final, RFtest_lst_final = [],[]

nrset = [2000,3000,5000,10000]

# curcol = 'duplicateColumn'
# MODEL_TYPE = 'xgb'

# PERC_ENTY = [10,30,50]
PERC_ENTY = [30]
PERC_OCCUR = 25

total_data_pts = []
LR_min_lst,LR_perc25_lst,LR_median_lst,LR_perc75_lst,LR_max_lst = [],[],[],[],[]
RF_min_lst,RF_perc25_lst,RF_median_lst,RF_perc75_lst,RF_max_lst = [],[],[],[],[]
inc_RF_min_lst,inc_RF_perc25_lst,inc_RF_median_lst,inc_RF_perc75_lst,inc_RF_max_lst = [],[],[],[],[]

gen_original_LR_final,gen_original_RF_final,gen_duplicate_LR_final,gen_duplicate_RF_final = [],[],[],[]
inc_var_LR_final,inc_var_RF_final = [],[]

NUMRUNS = 100
dxrval = 10
# a=[10]
totnos = 1
for i in PERC_ENTY:
#     try:
#     file0 = open('logs/synthetic/allxRF/duplicates-1(dr=3)/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')
#     file1 = open('logs/synthetic/allxRF/duplicates-1(dr=3)/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[1]) + ')-' + str(i) + '.txt', 'r')
    
#     file_lst = [file0,file1]
#     file_lst = [file0, file1, file2, file3, file4]
    
    file_lst = []
    for dumpq in nrset:
        f=open('logs/'+MODEL_TYPE+ '/duplicates-1(dr=3)/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(dumpq) + ')-' + str(PERC_ENTY[0]) + '.txt', 'r')
        file_lst.append(f)
    
    if ish2o: getFileInfoH2o(file_lst)
    elif ismlp: getFileInfoMLP(file_lst)
    else: getFileInfo(file_lst)
    

plt.figure(figsize=(8, 8))        
df = pd.DataFrame()
df['# train examples'] = nrset
# df['train /domain^3'] = trarr_3

if ismlp:
    plt.boxplot(LRtest_lst_final)
    
    plt.yticks([-6,-3,0,3,6,9])
    plt.xticks([i+1 for i in range(len(nrset))], nrset)
    plt.show()
    # plt.savefig('images/RF-allx-varyExamples.png', dpi=150)  
    plt.close()

    df['RF min'] = LR_min_lst
    df['RF 25th percentile'] = LR_perc25_lst
    df['RF median'] = LR_median_lst
    df['RF 75th percentile'] = LR_perc75_lst
    df['RF max'] = LR_max_lst
else:
    plt.boxplot(inc_var_RF_final)
    plt.xticks([i+1 for i in range(len(nrset))], nrset)
    plt.show()
#     plt.savefig('images/RFBias-allx-varyExamples.png', dpi=150)
    plt.close()

    df['RF min'] = RF_min_lst
    df['RF 25th percentile'] = RF_perc25_lst
    df['RF median'] = RF_median_lst
    df['inc_RF median'] = inc_RF_median_lst
    df['RF 75th percentile'] = RF_perc75_lst
    df['RF max'] = RF_max_lst

display(df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap',}))
    
    

# Vary Entities

In [ ]:
LRtest_lst_final, RFtest_lst_final = [],[]

nrset = [3000]

PERC_ENTY = [10,30,50,100]
PERC_OCCUR = 25

total_data_pts = []
LR_min_lst,LR_perc25_lst,LR_median_lst,LR_perc75_lst,LR_max_lst = [],[],[],[],[]
RF_min_lst,RF_perc25_lst,RF_median_lst,RF_perc75_lst,RF_max_lst = [],[],[],[],[]
inc_RF_min_lst,inc_RF_perc25_lst,inc_RF_median_lst,inc_RF_perc75_lst,inc_RF_max_lst = [],[],[],[],[]

gen_original_LR_final,gen_original_RF_final,gen_duplicate_LR_final,gen_duplicate_RF_final = [],[],[],[]
inc_var_LR_final,inc_var_RF_final = [],[]

NUMRUNS = 100
dxrval = 10

totnos = 1
for i in ['dummy']:
#     try:
#     file0 = open('logs/synthetic/allxRF/duplicates-1(dr=3)/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[0]) + '.txt', 'r')
#     file1 = open('logs/synthetic/allxRF/duplicates-1(dr=3)/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[1]) + '.txt', 'r')
    
    file_lst = []
    for dumpq in PERC_ENTY:
        f=open('logs/'+MODEL_TYPE+'/duplicates-1(dr=3)/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(dumpq) + '.txt', 'r')
        file_lst.append(f)
    
    if ish2o: getFileInfoH2o(file_lst)
    elif ismlp: getFileInfoMLP(file_lst)
    else: getFileInfo(file_lst)


plt.figure(figsize=(6, 6))        
df = pd.DataFrame()
df['# train examples'] = PERC_ENTY


if ismlp:
#     plt.boxplot(LRtest_lst_final)
    plt.boxplot(inc_var_LR_final)      
#     plt.yticks([-8,-6,-4,-2,0,2,4,6,8])
#     plt.yticks([0,2,4,6])
#     plt.yticks([-9,-6,-3,0,3,6,9])
    plt.xticks([i+1 for i in range(len(PERC_ENTY))], PERC_ENTY)
    # plt.show()
    # plt.savefig('images/inc-MLPBias-allx-varyEntities.png', dpi=150)      
    plt.savefig('images/XGB-allx-varyEntities.png', dpi=150)
    plt.close()

    df['RF min'] = LR_min_lst
    df['RF 25th percentile'] = LR_perc25_lst
    df['RF median'] = LR_median_lst
    df['inc_RF median'] = inc_RF_median_lst    
    df['RF 75th percentile'] = LR_perc75_lst
    df['RF max'] = LR_max_lst
else:
    plt.boxplot(inc_var_RF_final)    
    plt.xticks([i+1 for i in range(len(PERC_ENTY))], PERC_ENTY)
#     plt.show()
    plt.close()

    df['RF min'] = RF_min_lst
    df['RF 25th percentile'] = RF_perc25_lst
    df['RF median'] = RF_median_lst
    df['inc_RF median'] = inc_RF_median_lst
    df['RF 75th percentile'] = RF_perc75_lst
    df['RF max'] = RF_max_lst

display(df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap',}))



# Vary Occurrences

In [ ]:
LRtest_lst_final, RFtest_lst_final = [],[]

nrset = [3000]

PERC_ENTY = [30]
PERC_OCCUR = [10,25,50]

total_data_pts = []
LR_min_lst,LR_perc25_lst,LR_median_lst,LR_perc75_lst,LR_max_lst = [],[],[],[],[]
RF_min_lst,RF_perc25_lst,RF_median_lst,RF_perc75_lst,RF_max_lst = [],[],[],[],[]
gen_original_LR_final,gen_original_RF_final,gen_duplicate_LR_final,gen_duplicate_RF_final = [],[],[],[]
inc_var_LR_final,inc_var_RF_final = [],[]
inc_RF_min_lst,inc_RF_perc25_lst,inc_RF_median_lst,inc_RF_perc75_lst,inc_RF_max_lst = [],[],[],[],[]

NUMRUNS = 100
dxrval = 10

totnos = 1
for i in ['dummy']:
    
    file_lst = []
    for dumpq in PERC_OCCUR:
        f=open('logs/synthetic/'+MODEL_TYPE+'/duplicates-1(dr=3)/Occurrence=' + str(dumpq) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[0]) + '.txt', 'r')
        file_lst.append(f)
    
    if ish2o: getFileInfoH2o(file_lst)
    elif ismlp: getFileInfoMLP(file_lst)
    else: getFileInfo(file_lst)
    

plt.figure(figsize=(8, 8))        
df = pd.DataFrame()
df['# train examples'] = PERC_OCCUR


if ismlp:
    plt.boxplot(LRtest_lst_final)
#     plt.yticks([0,2,4,6])
    plt.yticks([-6,-3,0,3,6,9])    
    plt.xticks([i+1 for i in range(len(PERC_OCCUR))], PERC_OCCUR)
    plt.show()
    plt.close()

    df['RF min'] = LR_min_lst
    df['RF 25th percentile'] = LR_perc25_lst
    df['RF median'] = LR_median_lst
    df['RF 75th percentile'] = LR_perc75_lst
    df['RF max'] = LR_max_lst
else:
#     plt.boxplot(RFtest_lst_final)
    plt.boxplot(inc_var_RF_final)    
#     plt.yticks([-10,-5,0,5,10])
#     plt.yticks([0,2,4])
    plt.xticks([i+1 for i in range(len(PERC_OCCUR))], PERC_OCCUR)
#     plt.show()
    # plt.savefig('images/inc-RFVar-allx-varyOccur.png', dpi=150)    
    plt.close()

    df['RF min'] = RF_min_lst
    df['RF 25th percentile'] = RF_perc25_lst
    df['RF median'] = RF_median_lst
#     df['inc_RF median'] = inc_RF_median_lst
    df['RF 75th percentile'] = RF_perc75_lst
    df['RF max'] = RF_max_lst

display(df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap',}))


# Vary Group Size

In [ ]:
LRtest_lst_final, RFtest_lst_final = [],[]

nrset = [3000]

PERC_ENTY = [30]
PERC_OCCUR = [25]
grp_no = [1,3,5,10]

total_data_pts = []
LR_min_lst,LR_perc25_lst,LR_median_lst,LR_perc75_lst,LR_max_lst = [],[],[],[],[]
RF_min_lst,RF_perc25_lst,RF_median_lst,RF_perc75_lst,RF_max_lst = [],[],[],[],[]
gen_original_LR_final,gen_original_RF_final,gen_duplicate_LR_final,gen_duplicate_RF_final = [],[],[],[]
inc_var_LR_final,inc_var_RF_final = [],[]
inc_RF_min_lst,inc_RF_perc25_lst,inc_RF_median_lst,inc_RF_perc75_lst,inc_RF_max_lst = [],[],[],[],[]

NUMRUNS = 100
dxrval = 10

totnos = 1
for i in ['dummy']:
    
    file_lst = []
    for dumpq in grp_no:
        f=open('logs/synthetic/'+MODEL_TYPE+'/duplicates-'+str(dumpq)+'(dr=3)/Occurrence=' + str(PERC_OCCUR[0]) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[0]) + '.txt', 'r')
        file_lst.append(f)
    
    if ish2o: getFileInfoH2o(file_lst)
    elif ismlp: getFileInfoMLP(file_lst)
    else: getFileInfo(file_lst)
    

plt.figure(figsize=(8, 8))        
df = pd.DataFrame()
df['# train examples'] = grp_no


if ismlp:
    plt.boxplot(LRtest_lst_final)
    plt.yticks([0,2,4,6,8])
    plt.yticks([-6,-3,0,3,6,9])
    plt.xticks([i+1 for i in range(len(grp_no))], grp_no)
#     plt.show()
    plt.close()

    df['RF min'] = LR_min_lst
    df['RF 25th percentile'] = LR_perc25_lst
    df['RF median'] = LR_median_lst
    df['RF 75th percentile'] = LR_perc75_lst
    df['RF max'] = LR_max_lst
else:
#     plt.boxplot(RFtest_lst_final)
    plt.boxplot(inc_var_RF_final)    
#     plt.yticks([-10,-5,0,5,10])
#     plt.yticks([-2,0,2,4,6,8])
    plt.xticks([i+1 for i in range(len(grp_no))], grp_no)
#     plt.show()
    plt.savefig('images/inc-RFVar-allx-varyGrpSize.png', dpi=150)
#     plt.savefig('images/H2oVar-allx-varyGrpSize.png', dpi=150)
    plt.close()

    df['RF min'] = RF_min_lst
    df['RF 25th percentile'] = RF_perc25_lst
    df['RF median'] = RF_median_lst
#     df['inc_RF median'] = inc_RF_median_lst    
    df['RF 75th percentile'] = RF_perc75_lst
    df['RF max'] = RF_max_lst

display(df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap',}))


# Vary Needle Skew in duplication parameters

In [ ]:
curcol = 'duplicateColumn'
# curcol = 'noiseColumn'
LRtest_lst_final, RFtest_lst_final = [],[]

nrset = [3000]

PERC_ENTY = [30]
PERC_OCCUR = 25

total_data_pts = []
LR_min_lst,LR_perc25_lst,LR_median_lst,LR_perc75_lst,LR_max_lst = [],[],[],[],[]
RF_min_lst,RF_perc25_lst,RF_median_lst,RF_perc75_lst,RF_max_lst = [],[],[],[],[]
gen_original_LR_final,gen_original_RF_final,gen_duplicate_LR_final,gen_duplicate_RF_final = [],[],[],[]
inc_var_LR_final,inc_var_RF_final = [],[]

NUMRUNS = 100
dxrval = 10

totnos = 1
for i in PERC_ENTY:
#     try:
    file0 = open('logs/synthetic/allxRF-Var/skew/NEEDLE_PROB=0.05/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')
    file1 = open('logs/synthetic/allxRF-Var/skew/NEEDLE_PROB=0.25/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')
    file2 = open('logs/synthetic/allxRF-Var/skew/NEEDLE_PROB=0.5/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')    
    file3 = open('logs/synthetic/allxRF-Var/skew/NEEDLE_PROB=0.75/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')
    file4 = open('logs/synthetic/allxRF-Var/skew/NEEDLE_PROB=0.95/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')    
    
    file_lst = [file0, file1, file2, file3, file4]
    
    index = 0
    for file in file_lst:
#         print('==============' + str(index) + ':' + str(nrset[index]) + '==============')
        LRtrain_lst, LRval_lst, LRtest_lst = [], [], []
        RFtrain_lst, RFval_lst, RFtest_lst = [], [], []        
        gen_original_LR,gen_original_RF,gen_duplicate_LR,gen_duplicate_RF = [],[],[],[]
        inc_var_LR,inc_var_RF = [],[]        
        
        data = file.read()

        x = data.split('\n')

        curindex = 4
        while curindex < 39*NUMRUNS:
#             print(curindex)
            try:
                scrtr_LR, scrval_LR, scrte_LR = float(x[curindex-2]), float(x[curindex-1]), float(x[curindex])
                scrtr_RF, scrval_RF, scrte_RF = float(x[curindex+2]),float(x[curindex+3]),float(x[curindex+4])

#                 curindex = curindex+1
                LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+15]),ast.literal_eval(x[curindex+16]),ast.literal_eval(x[curindex+17])
                RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+19]),ast.literal_eval(x[curindex+20]),ast.literal_eval(x[curindex+21])

                gen_error_LR, gen_error_RF = (scrtr_LR-scrval_LR),(scrtr_RF-scrval_RF)
                gen_original_LR.extend([gen_error_LR])
                gen_original_RF.extend([gen_error_RF])                
        
#                 print('***' + str(len(RFtest)))
#                 print(RFtest)
    #             LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+25]),ast.literal_eval(x[curindex+26]),ast.literal_eval(x[curindex+27])
    #             print(ast.literal_eval(x[curindex+28]))
    #             RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+29]),ast.literal_eval(x[curindex+30]),ast.literal_eval(x[curindex+31])

                diff_train_lst_LR_full = [round(scrtr_LR - number,3) for number in LRtrain]
                diff_lst_LR_full = [round(scrval_LR - number,3) for number in LRval] 
                diff_hld_lst_LR_full = [round(scrte_LR - number,3) for number in LRtest]

                diff_train_lst_RF_full = [round(scrtr_RF - number,3) for number in RFtrain]
                diff_lst_RF_full = [round(scrval_RF - number,3) for number in RFval]
                diff_hld_lst_RF_full = [round(scrte_RF - number,3) for number in RFtest]


                LRtrain_lst.extend(diff_train_lst_LR_full)
                LRval_lst.extend(diff_lst_LR_full)
                LRtest_lst.extend(diff_hld_lst_LR_full)

                RFtrain_lst.extend(diff_train_lst_RF_full)
                RFval_lst.extend(diff_lst_RF_full)
                RFtest_lst.extend(diff_hld_lst_RF_full)
                
                gen_duplicate_LR.extend([round(LRtrain[pq] - LRval[pq],3) for pq in range(len(LRtrain))])
                gen_duplicate_RF.extend([round(RFtrain[pq] - RFval[pq],3) for pq in range(len(RFtrain))])                

                inc_var_LR.extend([round(LRtrain[pq] - LRval[pq]-gen_error_LR,3) for pq in range(len(LRtrain))])
                inc_var_RF.extend([round(RFtrain[pq] - RFval[pq]-gen_error_RF,3) for pq in range(len(RFtrain))])
                                
            except: flag = 1
            curindex = curindex + 39                
            
        LRtest_lst_final.append(LRtest_lst)
        RFtest_lst_final.append(RFtest_lst)

        gen_original_LR_final.append(gen_original_LR)
        gen_original_RF_final.append(gen_original_RF)
        gen_duplicate_LR_final.append(gen_duplicate_LR)
        gen_duplicate_RF_final.append(gen_duplicate_RF)        
        inc_var_LR_final.append(inc_var_LR)
        inc_var_RF_final.append(inc_var_RF)        
        
#         print(len(LRtest_lst_final))
#         print(totnos)
        print(len(RFtest_lst_final[index]))
        totnos += 1
        
        RF_min_lst.append(np.min(RFtest_lst_final[index]))
        RF_perc25_lst.append(np.quantile(RFtest_lst_final[index], .25))
        RF_median_lst.append(np.median(RFtest_lst_final[index]))
        RF_perc75_lst.append(np.quantile(RFtest_lst_final[index], .75))
        RF_max_lst.append(np.max(RFtest_lst_final[index]))
        
        index = index +1

tmparr = [0.05,0.25,0.5,0.75,0.95]
plt.figure(figsize=(8, 8))
# plt.ylabel('Lift with Cleaning relative to the dirty dataset')
# plt.xlabel(curcol + ' LR')
plt.boxplot(LRtest_lst_final)
# plt.yticks([-4,-2,0,2,4,6,8])
plt.xticks([i+1 for i in range(len(tmparr))], tmparr)
# plt.show()
# plt.savefig('LR-allx.png', dpi=400)
plt.close()

plt.figure(figsize=(8, 8))
# plt.ylabel('Lift with Cleaning relative to the dirty dataset')
# plt.xlabel(curcol + ' RF')
plt.boxplot(RFtest_lst_final)
# plt.yticks([-4,-2,0,2,4,6,8])
# plt.xticks([i+1 for i in range(len([0,2,4]))], [0,2,4])
plt.xticks([i+1 for i in range(len(tmparr))], tmparr)
# plt.show()
plt.savefig('images/RF-allx-NeedleSkew.png', dpi=150)
plt.close()

trarr_2 = [x/(dxrval*dxrval) for x in nrset]
trarr_3 = [x/(dxrval*dxrval*dxrval) for x in nrset]

df = pd.DataFrame()

df['RF min'] = RF_min_lst
df['RF 25th percentile'] = RF_perc25_lst
df['RF median'] = RF_median_lst
df['RF 75th percentile'] = RF_perc75_lst
df['RF max'] = RF_max_lst
df


# Vary Zipfian Skew in duplication parameters

In [ ]:
curcol = 'duplicateColumn'
# curcol = 'noiseColumn'
LRtest_lst_final, RFtest_lst_final = [],[]

nrset = [3000]

PERC_ENTY = [30]
PERC_OCCUR = 25

total_data_pts = []
LR_min_lst,LR_perc25_lst,LR_median_lst,LR_perc75_lst,LR_max_lst = [],[],[],[],[]
RF_min_lst,RF_perc25_lst,RF_median_lst,RF_perc75_lst,RF_max_lst = [],[],[],[],[]
gen_original_LR_final,gen_original_RF_final,gen_duplicate_LR_final,gen_duplicate_RF_final = [],[],[],[]
inc_var_LR_final,inc_var_RF_final = [],[]

NUMRUNS = 100
dxrval = 10

totnos = 1
for i in PERC_ENTY:
#     try:
    file0 = open('logs/synthetic/allxRF-Var/skew/zipf=0/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')
    file1 = open('logs/synthetic/allxRF-Var/skew/zipf=2/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')
    file2 = open('logs/synthetic/allxRF-Var/skew/zipf=4/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(i) + '.txt', 'r')    

    file_lst = [file0, file1, file2]
    
    index = 0
    for file in file_lst:
#         print('==============' + str(index) + ':' + str(nrset[index]) + '==============')
        LRtrain_lst, LRval_lst, LRtest_lst = [], [], []
        RFtrain_lst, RFval_lst, RFtest_lst = [], [], []        
        gen_original_LR,gen_original_RF,gen_duplicate_LR,gen_duplicate_RF = [],[],[],[]
        inc_var_LR,inc_var_RF = [],[]        
        
        data = file.read()

        x = data.split('\n')

        curindex = 4
        while curindex < 39*NUMRUNS:
#             print(curindex)
            try:
                scrtr_LR, scrval_LR, scrte_LR = float(x[curindex-2]), float(x[curindex-1]), float(x[curindex])
                scrtr_RF, scrval_RF, scrte_RF = float(x[curindex+2]),float(x[curindex+3]),float(x[curindex+4])

#                 curindex = curindex+1
                LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+15]),ast.literal_eval(x[curindex+16]),ast.literal_eval(x[curindex+17])
                RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+19]),ast.literal_eval(x[curindex+20]),ast.literal_eval(x[curindex+21])

                gen_error_LR, gen_error_RF = (scrtr_LR-scrval_LR),(scrtr_RF-scrval_RF)
                gen_original_LR.extend([gen_error_LR])
                gen_original_RF.extend([gen_error_RF])                
        
#                 print('***' + str(len(RFtest)))
#                 print(RFtest)
    #             LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+25]),ast.literal_eval(x[curindex+26]),ast.literal_eval(x[curindex+27])
    #             print(ast.literal_eval(x[curindex+28]))
    #             RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+29]),ast.literal_eval(x[curindex+30]),ast.literal_eval(x[curindex+31])

                diff_train_lst_LR_full = [round(scrtr_LR - number,3) for number in LRtrain]
                diff_lst_LR_full = [round(scrval_LR - number,3) for number in LRval] 
                diff_hld_lst_LR_full = [round(scrte_LR - number,3) for number in LRtest]

                diff_train_lst_RF_full = [round(scrtr_RF - number,3) for number in RFtrain]
                diff_lst_RF_full = [round(scrval_RF - number,3) for number in RFval]
                diff_hld_lst_RF_full = [round(scrte_RF - number,3) for number in RFtest]


                LRtrain_lst.extend(diff_train_lst_LR_full)
                LRval_lst.extend(diff_lst_LR_full)
                LRtest_lst.extend(diff_hld_lst_LR_full)

                RFtrain_lst.extend(diff_train_lst_RF_full)
                RFval_lst.extend(diff_lst_RF_full)
                RFtest_lst.extend(diff_hld_lst_RF_full)
                
                gen_duplicate_LR.extend([round(LRtrain[pq] - LRval[pq],3) for pq in range(len(LRtrain))])
                gen_duplicate_RF.extend([round(RFtrain[pq] - RFval[pq],3) for pq in range(len(RFtrain))])                

                inc_var_LR.extend([round(LRtrain[pq] - LRval[pq]-gen_error_LR,3) for pq in range(len(LRtrain))])
                inc_var_RF.extend([round(RFtrain[pq] - RFval[pq]-gen_error_RF,3) for pq in range(len(RFtrain))])
                                
            except: flag = 1
            curindex = curindex + 39                
            
        LRtest_lst_final.append(LRtest_lst)
        RFtest_lst_final.append(RFtest_lst)

        gen_original_LR_final.append(gen_original_LR)
        gen_original_RF_final.append(gen_original_RF)
        gen_duplicate_LR_final.append(gen_duplicate_LR)
        gen_duplicate_RF_final.append(gen_duplicate_RF)        
        inc_var_LR_final.append(inc_var_LR)
        inc_var_RF_final.append(inc_var_RF)        
        
#         print(len(LRtest_lst_final))
#         print(totnos)
        print(len(RFtest_lst_final[index]))
        totnos += 1
        
#         total_data_pts.append(len(LRtest_lst_final[index]))
#         LR_min_lst.append(np.min(LRtest_lst_final[index]))
#         LR_perc25_lst.append(np.quantile(LRtest_lst_final[index], .25))
#         LR_median_lst.append(np.median(LRtest_lst_final[index]))
#         LR_perc75_lst.append(np.quantile(LRtest_lst_final[index], .75))
#         LR_max_lst.append(np.max(LRtest_lst_final[index]))
        
        RF_min_lst.append(np.min(RFtest_lst_final[index]))
        RF_perc25_lst.append(np.quantile(RFtest_lst_final[index], .25))
        RF_median_lst.append(np.median(RFtest_lst_final[index]))
        RF_perc75_lst.append(np.quantile(RFtest_lst_final[index], .75))
        RF_max_lst.append(np.max(RFtest_lst_final[index]))
        
        index = index +1
        
plt.figure(figsize=(8, 8))
# plt.ylabel('Lift with Cleaning relative to the dirty dataset')
# plt.xlabel(curcol + ' LR')
plt.boxplot(LRtest_lst_final)
# plt.yticks([-4,-2,0,2,4,6,8])
plt.xticks([i+1 for i in range(len([0,2,4]))], [0,2,4])
# plt.show()
# plt.savefig('LR-allx.png', dpi=400)
plt.close()

plt.figure(figsize=(8, 8))
# plt.ylabel('Lift with Cleaning relative to the dirty dataset')
# plt.xlabel(curcol + ' RF')
plt.boxplot(RFtest_lst_final)
# plt.yticks([-4,-2,0,2,4,6,8])
plt.xticks([i+1 for i in range(len([0,2,4]))], [0,2,4])
# plt.show()
plt.savefig('images/RF-allx-ZipSkew.png', dpi=150)
# plt.savefig('images/RF-allx-varyExamples.png', dpi=150)
plt.close()

trarr_2 = [x/(dxrval*dxrval) for x in nrset]
trarr_3 = [x/(dxrval*dxrval*dxrval) for x in nrset]

df = pd.DataFrame()
# df['# train examples'] = nrset
# df['train /domain^2'] = trarr_2
# df['train /domain^3'] = trarr_3
# df['total data points'] = total_data_pts
# df['LR min'] = LR_min_lst
# df['LR 25th percentile'] = LR_perc25_lst
# df['LR median'] = LR_median_lst
# df['LR 75th percentile'] = LR_perc75_lst
# df['LR max'] = LR_max_lst

df['RF min'] = RF_min_lst
df['RF 25th percentile'] = RF_perc25_lst
df['RF median'] = RF_median_lst
df['RF 75th percentile'] = RF_perc75_lst
df['RF max'] = RF_max_lst
df


# Vary Column Relevancy

In [ ]:
curcol = 'duplicateColumn'
curcol = 'noiseColumn'
LRtest_lst_final, RFtest_lst_final = [],[]

nrset = [3000]
PERC_ENTY = [10,30,50]
PERC_OCCUR = 25

total_data_pts = []
LR_min_lst,LR_perc25_lst,LR_median_lst,LR_perc75_lst,LR_max_lst = [],[],[],[],[]
RF_min_lst,RF_perc25_lst,RF_median_lst,RF_perc75_lst,RF_max_lst = [],[],[],[],[]

NUMRUNS = 100
dxrval = 10
dr = 4

totnos = 1
for i in nrset:
#     try:
    file0 = open('logs/synthetic/allxRF-Var/duplicates-1(dr=' + str(dr) + ')/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[0]) + '.txt', 'r')
    file1 = open('logs/synthetic/allxRF-Var/duplicates-1(dr=' + str(dr) + ')/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[1]) + '.txt', 'r')
    file2 = open('logs/synthetic/allxRF-Var/duplicates-1(dr=' + str(dr) + ')/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[2]) + '.txt', 'r')
#     file3 = open('logs/synthetic/allxRF-Var/duplicates-1(dr=' + str(dr) + ')/Occurrence=' + str(PERC_OCCUR) + '/' + str(curcol) + '(dxr=' + str(dxrval) + ')(nr=' + str(nrset[0]) + ')-' + str(PERC_ENTY[3]) + '.txt', 'r')    
#     file_lst = [file0, file1, file2, file3, file4]
#     file_lst = [file0, file1, file2, file3]    
    file_lst = [file0, file1, file2]
#     file_lst = [file0]    
    
    index = 0
    for file in file_lst:
#         print('==============' + str(index) + ':' + str(nrset[index]) + '==============')
        LRtrain_lst, LRval_lst, LRtest_lst = [], [], []
        RFtrain_lst, RFval_lst, RFtest_lst = [], [], []        
        
        data = file.read()

        x = data.split('\n')

        curindex = 4
        while curindex < 39*NUMRUNS:
#             print(curindex)
            try:
                scrtr_LR, scrval_LR, scrte_LR = float(x[curindex-2]), float(x[curindex-1]), float(x[curindex])
                scrtr_RF, scrval_RF, scrte_RF = float(x[curindex+2]),float(x[curindex+3]),float(x[curindex+4])

                curindex = curindex+ (dr-3)
                LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+15]),ast.literal_eval(x[curindex+16]),ast.literal_eval(x[curindex+17])
                RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+19]),ast.literal_eval(x[curindex+20]),ast.literal_eval(x[curindex+21])

#                 print('***' + str(len(RFtest)))
#                 print(RFtest)
    #             LRtrain, LRval, LRtest = ast.literal_eval(x[curindex+25]),ast.literal_eval(x[curindex+26]),ast.literal_eval(x[curindex+27])
    #             print(ast.literal_eval(x[curindex+28]))
    #             RFtrain, RFval, RFtest = ast.literal_eval(x[curindex+29]),ast.literal_eval(x[curindex+30]),ast.literal_eval(x[curindex+31])

                diff_train_lst_LR_full = [round(scrtr_LR - number,3) for number in LRtrain]
                diff_lst_LR_full = [round(scrval_LR - number,3) for number in LRval] 
                diff_hld_lst_LR_full = [round(scrte_LR - number,3) for number in LRtest]

                diff_train_lst_RF_full = [round(scrtr_RF - number,3) for number in RFtrain]
                diff_lst_RF_full = [round(scrval_RF - number,3) for number in RFval]
                diff_hld_lst_RF_full = [round(scrte_RF - number,3) for number in RFtest]


                LRtrain_lst.extend(diff_train_lst_LR_full)
                LRval_lst.extend(diff_lst_LR_full)
                LRtest_lst.extend(diff_hld_lst_LR_full)

                RFtrain_lst.extend(diff_train_lst_RF_full)
                RFval_lst.extend(diff_lst_RF_full)
                RFtest_lst.extend(diff_hld_lst_RF_full)
            except: flag = 1
            curindex = curindex + 39                
            
        LRtest_lst_final.append(LRtest_lst)
        RFtest_lst_final.append(RFtest_lst)

#         print(len(LRtest_lst_final))
#         print(totnos)
        print(len(RFtest_lst_final[index]))
        totnos += 1
        
        
        RF_min_lst.append(np.min(RFtest_lst_final[index]))
        RF_perc25_lst.append(np.quantile(RFtest_lst_final[index], .25))
        RF_median_lst.append(np.median(RFtest_lst_final[index]))
        RF_perc75_lst.append(np.quantile(RFtest_lst_final[index], .75))
        RF_max_lst.append(np.max(RFtest_lst_final[index]))

        index = index +1
        
# for i in range(len(RFtest_lst_final)):
#     for j in range(len(RFtest_lst_final[i])):
#         if RFtest_lst_final[i][j] < -0.5:
#             RFtest_lst_final[i][j] = RFtest_lst_final[i][j]+1


plt.figure(figsize=(8,8))
# plt.ylabel('Lift with Cleaning relative to the dirty dataset')
# plt.xlabel(curcol + ' RF')
plt.boxplot(RFtest_lst_final)
plt.xticks([i+1 for i in range(len(PERC_ENTY))], PERC_ENTY)
# plt.show()
# plt.savefig('images/RF-allx-relevancy-dupCol.png', dpi=150)
plt.savefig('images/RF-allx-relevancy-noiseCol.png', dpi=150)
plt.close()

trarr_2 = [x/(dxrval*dxrval) for x in nrset]
trarr_3 = [x/(dxrval*dxrval*dxrval) for x in nrset]

df = pd.DataFrame()
df['LR min'] = LR_min_lst
df['LR 25th percentile'] = LR_perc25_lst
df['LR median'] = LR_median_lst
df['LR 75th percentile'] = LR_perc75_lst
df['LR max'] = LR_max_lst

df['RF min'] = RF_min_lst
df['RF 25th percentile'] = RF_perc25_lst
df['RF median'] = RF_median_lst
df['RF 75th percentile'] = RF_perc75_lst
df['RF max'] = RF_max_lst
df
